In [ ]:
# !pip install datasets #for google collab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [ ]:
from datasets import load_dataset

ds = load_dataset("RZ412/PokerBench")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.16k [00:00<?, ?B/s]

(…)lop_500k_train_set_prompt_and_label.json:   0%|          | 0.00/607M [00:00<?, ?B/s]

(…)flop_60k_train_set_prompt_and_label.json:   0%|          | 0.00/62.0M [00:00<?, ?B/s]

(…)tflop_10k_test_set_prompt_and_label.json:   0%|          | 0.00/12.2M [00:00<?, ?B/s]

(…)reflop_1k_test_set_prompt_and_label.json:   0%|          | 0.00/965k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/563200 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11000 [00:00<?, ? examples/s]

In [71]:
from tqdm import tqdm

print(ds)
i = 500
prompt = ds["train"]["instruction"][i]
print(prompt)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 563200
    })
    test: Dataset({
        features: ['instruction', 'output'],
        num_rows: 11000
    })
})


You are a specialist in playing 6-handed No Limit Texas Holdem. The following will be a game scenario and you need to make the optimal decision.

Here is a game summary:

The small blind is 0.5 chips and the big blind is 1 chips. Everyone started with 100 chips.
The player positions involved in this game are UTG, HJ, CO, BTN, SB, BB.
In this hand, your position is BB, and your holding is [Eight of Club and Seven of Spade].
Before the flop, HJ raise 2.0 chips, and BB call. Assume that all other players that is not mentioned folded.
The flop comes Five Of Club, Five Of Diamond, and Ace Of Heart, then BB check, and HJ check.
The turn comes Five Of Spade, then BB bet 5 chips, and HJ call.
The river comes Ten Of Diamond, then BB check, and HJ bet 11 chips.
You currently have Three 

In [76]:
def find_hands(prompt, start = 0):
  return prompt.find("[",start)
def find_flop(prompt, start = 0):
  return prompt.find("The flop comes",start)
def find_turn(prompt, start = 0):
  return prompt.find("The turn comes",start)
def find_river(prompt, start = 0):
  return prompt.find("The river comes",start)

def word_to_suit(word):
  if "Spade" in word:
    return 's'
  if "Club" in word:
    return 'c'
  if "Diamond" in word:
    return 'd'
  if "Heart" in word:
    return 'h'
  return "default"
def word_to_number(word):
  if "Ace" in word:
    return '1'
  if "Two" in word:
    return '2'
  if "Three" in word:
    return '3'
  if "Four" in word:
    return '4'
  if "Five" in word:
    return '5'
  if "Six" in word:
    return '6'
  if "Seven" in word:
    return '7'
  if "Eight" in word:
    return '8'
  if "Nine" in word:
    return '9'
  if "Ten" in word:
    return '10'
  if "Jack" in word:
    return 'J'
  if "Queen" in word:
    return 'Q'
  if "King" in word:
    return 'K'
  return "default"

def fill_flop_turn_river(prompt,start):
  flop = None
  turn = None
  river = None
  if find_flop(prompt,start) != -1:
    start = find_flop(prompt,start) + len("The flop comes")
    end = prompt.find("then", start) -2
    card_list = prompt[start : end].split(",")
    #Make hands
    flop = []
    for word in card_list:
      card =  word_to_number(word) + word_to_suit(word)
      flop.append(card)
    # print(flop)
    start = end

    if find_turn(prompt,start) != -1 :
      start = find_turn(prompt,start) + len("The turn comes")
      end = prompt.find("then", start) -2
      card_list = prompt[start : end].split(",")
      #Make hands
      turn = []
      for word in card_list:
        card =  word_to_number(word) + word_to_suit(word)
        turn.append(card)
      # print(turn)
      start = end

      if find_river(prompt,start) != -1:
        start = find_river(prompt,start) + len("The river comes")
        end = prompt.find("then", start) -2
        card_list = prompt[start : end].split(",")
        #Make hands
        river = []
        for word in card_list:
          card =  word_to_number(word) + word_to_suit(word)
          river.append(card)
        # print(river)
  next_index = start
  return flop,turn,river, next_index
# data_structure:
#   hands :
#   flop : card1, card2, card3
#   turn :
#   river :
#   pot size: $?
#   your chips : $?

hands = None
flop = None
turn = None
river = None
pot_size = None
your_chips = None

if find_hands(prompt) != -1:
  #find string
  start = find_hands(prompt) + 1
  end = prompt.find(']',start)
  card_list = prompt[start:end].split("and")
  #Make hands
  hands = []
  for word in card_list:
    card =  word_to_number(word) + word_to_suit(word)
    hands.append(card)
  #update next search pos
  start = end
  flop, turn, river, start = fill_flop_turn_river(prompt, start)

  start = prompt.find("the current pot size is ", start) + len("the current pot size is ")
  end = prompt.find(" chips", start)
  pot_size = float(prompt[start:end])
  print(hands, flop, turn, river,pot_size)

['8c', '7s'] ['5c', '5d', '1h'] ['5s'] ['10d']
